In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

## Function Overview: `classify_time_of_day`

**Purpose**:
- Classifies a given timestamp into a time of day category based on the hour.

**Parameters**:
- `timestamp`: A datetime object containing the time information.

**Returns**:
- `1` for morning (4 AM to 11:59 AM).
- `2` for afternoon to evening (12 PM to 7:59 PM).
- `3` for night (8 PM to 3:59 AM).

**Usage**:
This function can be used to categorize activities or events into time-of-day segments for analysis or reporting.


In [2]:
def classify_time_of_day(timestamp):
    hour = timestamp.hour
    if 4 <= hour < 12:
        return 1
    elif 12 <= hour < 20:
        return 2
    else:
        return 3

## Data Handling Process

1. **Load Metadata**:
   - The metadata is loaded from `id_metadata.csv`, using tab (`'\t'`) as the delimiter, into the `metadata_df` DataFrame.

2. **Load Listening History**:
   - Listening history data is loaded from `lh_reduced.csv` into the `df` DataFrame.

3. **Convert Timestamps**:
   - The `timestamp` column in `df` is converted to datetime format to facilitate time-based operations.

4. **Rename Columns**:
   - In `metadata_df`, the column named `id` is renamed to `song` to align with the naming in `df`, facilitating easier merging or joining operations in future steps.

5. **Classify Time of Day**:
   - The `timestamp` column in `df` is used to apply the `classify_time_of_day` function, which categorizes timestamps into three time of day segments (morning, afternoon to evening, and night). The result is stored in a new column `time_of_day`.

6. **Output DataFrame**:
   - The final DataFrame `df`, now enhanced with the `time_of_day` categorization, is displayed or utilized for subsequent analysis.


In [3]:
metadata_path = '../data/id_metadata.csv'
listening_history_path = '../data/listening_history.csv'
metadata_df = pd.read_csv(metadata_path, delimiter='\t')
df = pd.read_csv(listening_history_path, delimiter='\t')

df['timestamp'] = pd.to_datetime(df['timestamp'])
metadata_df.rename(columns={'id': 'song'}, inplace=True)
df['time_of_day'] = df['timestamp'].apply(classify_time_of_day)
df

,user,song,timestamp,time_of_day
0,user_007XIjOr,DaTQ53TUmfP93FSr,2019-02-20 12:28:00,2
1,user_007XIjOr,dGeyvi5WCOjDU7da,2019-02-20 12:35:00,2
2,user_007XIjOr,qUm54NYOjeFhmKYx,2019-02-20 12:48:00,2
3,user_007XIjOr,FtnuMT1DlevSR2n5,2019-02-20 12:52:00,2
4,user_007XIjOr,LHETTZcSZLeaVOGh,2019-02-20 13:09:00,2
...,...,...,...,...
5109587,user_zzWscYTy,BBiswLufo26YQCT7,2019-01-10 15:57:00,2
5109588,user_zzWscYTy,5ZHgff3sjETIiedr,2019-01-10 16:21:00,2
5109589,user_zzWscYTy,m4O1iLh6fC43xjRy,2019-01-10 16:48:00,2
5109590,user_zzWscYTy,mvUaP8k67qOFfA65,2019-01-10 21:13:00,3


## Data Filtering Process Based on Recent Dates

1. **Determine Latest Date**:
   - Calculate the most recent date (`latest_date`) in the `timestamp` column of the DataFrame `df`.

2. **Compute Date for One Week Ago**:
   - Subtract 7 days from the `latest_date` using `pd.Timedelta`, resulting in the date `one_week_ago`.

3. **Filter Recent Data**:
   - Restrict `df` to only include rows where the `timestamp` is on or after `one_week_ago`, effectively filtering the data to the last week.

4. **Output Filtered DataFrame**:
   - The resulting DataFrame `df` now contains only the records from the past week, ready for analysis or further processing.


In [4]:
latest_date = df['timestamp'].max()
one_week_ago = latest_date - pd.Timedelta(days=7)
df = df[df['timestamp'] >= one_week_ago]
df

,user,song,timestamp,time_of_day
5288,user_02jFGVkG,ke5JKpLa8Dw7dCDO,2019-03-19 12:39:00,2
5289,user_02jFGVkG,nRpEFcFzYZ8Z8Cye,2019-03-19 12:44:00,2
5290,user_02jFGVkG,UTDxdZ1outySsU7O,2019-03-19 15:18:00,2
5291,user_02jFGVkG,UTDxdZ1outySsU7O,2019-03-19 15:24:00,2
5292,user_02jFGVkG,bxo3drSzBGDlrodp,2019-03-19 15:24:00,2
...,...,...,...,...
5104162,user_zwQunLVn,A8GhNkPLuKusMIvF,2019-03-21 18:29:00,2
5104163,user_zwQunLVn,0m1MyuwLx1EjuQqD,2019-03-21 22:29:00,3
5104164,user_zwQunLVn,0m1MyuwLx1EjuQqD,2019-03-21 22:33:00,3
5104165,user_zwQunLVn,l2kISVY4j8iz1Gg7,2019-03-21 22:36:00,3


## Data Processing 

1. **Merge DataFrames**:
   - `df` is merged with selected columns from `metadata_df` based on the 'song' column. The selected columns include song attributes such as release year, popularity, danceability, energy, key, mode, valence, and tempo.

2. **Standardize Numeric Data**:
   - Identify numeric columns in the merged DataFrame using `select_dtypes`.
   - Standardize these numeric columns using `StandardScaler` to normalize the data, aiding in model performance and stability.


In [5]:
df = pd.merge(df, metadata_df[
    ['song', 'release', 'popularity', 'danceability', 'energy', 'key', 'mode', 'valence', 'tempo']], on='song')

numeric_cols_df = df.select_dtypes(include=np.number).columns
sscaler = StandardScaler()
df[numeric_cols_df] = sscaler.fit_transform(df[numeric_cols_df])
df

,user,song,timestamp,time_of_day,release,popularity,danceability,energy,key,mode,valence,tempo
0,user_02jFGVkG,ke5JKpLa8Dw7dCDO,2019-03-19 12:39:00,-0.142363,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170
1,user_2uotd4qv,ke5JKpLa8Dw7dCDO,2019-03-19 21:49:00,1.212425,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170
2,user_McRU0r8i,ke5JKpLa8Dw7dCDO,2019-03-20 18:59:00,-0.142363,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170
3,user_McRU0r8i,ke5JKpLa8Dw7dCDO,2019-03-20 19:41:00,-0.142363,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170
4,user_02jFGVkG,nRpEFcFzYZ8Z8Cye,2019-03-19 12:44:00,-0.142363,-1.482494,0.239512,0.021888,0.787531,0.482058,-1.250220,0.562468,-1.106867
...,...,...,...,...,...,...,...,...,...,...,...,...
68085,user_zwQunLVn,VnF0TlGM24ZQaFxu,2019-03-21 11:02:00,-1.497152,0.630480,-0.065130,-0.226151,-0.940568,-1.195683,0.799859,-0.877934,0.236975
68086,user_zwQunLVn,7ViolZVDtOxSjByn,2019-03-21 12:30:00,-0.142363,0.630480,-0.186987,0.567573,1.355406,1.041305,-1.250220,0.967719,-0.926846
68087,user_zwQunLVn,Bk8GOanz19kxdO59,2019-03-21 18:02:00,-0.142363,0.630480,-0.369772,0.424951,1.272183,0.482058,0.799859,1.778221,0.341861
68088,user_zwQunLVn,qUXY2aw3nHQ4Qslh,2019-03-21 18:06:00,-0.142363,0.342347,-0.369772,-0.288160,0.748368,-1.195683,-1.250220,-0.666499,-0.400794


## DataFrame Operations Overview

1. **Check for Missing Values**:
   - `df.isnull().sum()` calculates the total number of missing values in each column of the DataFrame.
   
2. **Identify Unique Entries**:
   - `df.song.unique()` retrieves an array of unique song IDs from the `song` column.
   - `df.user.unique()` retrieves an array of unique user IDs from the `user` column.

3. **Calculate Song Popularity**:
   - The popularity of each song is calculated as the frequency of the song's appearance in the DataFrame divided by the total number of unique songs.
   - This calculated popularity is then mapped back to the `song` column of `df` and stored in a new column `song_popularity`.

The final DataFrame `df` is enhanced with a new `song_popularity` column which provides a relative measure of how frequently each song appears in the dataset, adjusted by the number of unique songs.


In [6]:
df.isnull().sum()

user            0
song            0
timestamp       0
time_of_day     0
release         0
popularity      0
danceability    0
energy          0
key             0
mode            0
valence         0
tempo           0
dtype: int64

In [7]:
unique_names_song = df.song.unique()
unique_names_user = df.user.unique()
unique_names_song.shape, unique_names_user.shape

((18193,), (633,))

In [8]:
df.isnull().values.any()
song_popularity = df['song'].value_counts() / len(unique_names_song)
df['song_popularity'] = df['song'].map(song_popularity)
df

,user,song,timestamp,time_of_day,release,popularity,danceability,energy,key,mode,valence,tempo,song_popularity
0,user_02jFGVkG,ke5JKpLa8Dw7dCDO,2019-03-19 12:39:00,-0.142363,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170,0.000220
1,user_2uotd4qv,ke5JKpLa8Dw7dCDO,2019-03-19 21:49:00,1.212425,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170,0.000220
2,user_McRU0r8i,ke5JKpLa8Dw7dCDO,2019-03-20 18:59:00,-0.142363,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170,0.000220
3,user_McRU0r8i,ke5JKpLa8Dw7dCDO,2019-03-20 19:41:00,-0.142363,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170,0.000220
4,user_02jFGVkG,nRpEFcFzYZ8Z8Cye,2019-03-19 12:44:00,-0.142363,-1.482494,0.239512,0.021888,0.787531,0.482058,-1.250220,0.562468,-1.106867,0.000715
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68085,user_zwQunLVn,VnF0TlGM24ZQaFxu,2019-03-21 11:02:00,-1.497152,0.630480,-0.065130,-0.226151,-0.940568,-1.195683,0.799859,-0.877934,0.236975,0.000055
68086,user_zwQunLVn,7ViolZVDtOxSjByn,2019-03-21 12:30:00,-0.142363,0.630480,-0.186987,0.567573,1.355406,1.041305,-1.250220,0.967719,-0.926846,0.000055
68087,user_zwQunLVn,Bk8GOanz19kxdO59,2019-03-21 18:02:00,-0.142363,0.630480,-0.369772,0.424951,1.272183,0.482058,0.799859,1.778221,0.341861,0.000055
68088,user_zwQunLVn,qUXY2aw3nHQ4Qslh,2019-03-21 18:06:00,-0.142363,0.342347,-0.369772,-0.288160,0.748368,-1.195683,-1.250220,-0.666499,-0.400794,0.000055


## Building Interaction Matrix and Enriched Data

### 1. **Initialize Interaction Matrix**:
   - An interaction matrix is created with dimensions corresponding to the unique count of users and songs, initialized with zeros.

### 2. **Map Users and Songs to Matrix Indices**:
   - Dictionaries `user_indices` and `song_indices` are created to map user and song identifiers to matrix indices for easy access.

### 3. **Populate Interaction Matrix**:
   - Iterate through the DataFrame `df`, using mapped indices to fill the matrix with the logarithm of song popularity incremented by one, to factor in popularity dynamics in interactions.

### 4. **Map Song Features**:
   - Extract and map song-related features like release date, popularity, danceability, and others into dictionaries from `df`, indexed by song ID.

### 5. **Prepare Data for Detailed Interaction DataFrame**:
   - Arrays are prepared for user IDs, song IDs, song features, and interaction values by iterating over the interaction matrix and mapping features for each song-user pair.

### 6. **Construct Feature-Rich DataFrame**:
   - A new DataFrame `interaction_df` is created to encapsulate user IDs, song IDs, their interactions, and all the additional song features such as release, popularity, and more.

This process effectively creates a structured and detailed view of user-song interactions, which is essential for tasks like recommendation systems or user behavior analysis based on music preferences.


In [9]:
interaction_matrix = np.zeros((df['user'].nunique(), len(unique_names_song)))

# Map users and songs to matrix indices
user_indices = {user: idx for idx, user in enumerate(df['user'].unique())}
song_indices = {song: idx for idx, song in enumerate(unique_names_song)}

for index, row in df.iterrows():
    user_idx = user_indices[row['user']]
    song_idx = song_indices[row['song']]
    interaction_matrix[user_idx, song_idx] = np.log(row['song_popularity'] + 1)

# Prepare dictionaries to map song IDs to their features
song_features = {
    'release': df.set_index('song')['release'].to_dict(),
    'popularity': df.set_index('song')['popularity'].to_dict(),
    'danceability': df.set_index('song')['danceability'].to_dict(),
    'energy': df.set_index('song')['energy'].to_dict(),
    'key': df.set_index('song')['key'].to_dict(),
    'mode': df.set_index('song')['mode'].to_dict(),
    'valence': df.set_index('song')['valence'].to_dict(),
    'tempo': df.set_index('song')['tempo'].to_dict(),
    'time_of_day': df.set_index('song')['time_of_day'].to_dict(),
}

# Create lists for DataFrame including additional features
user_ids, song_ids, releases, popularities, danceabilities, energies, keys, modes, valences, tempos, interactions, time_of_days = [], [], [], [], [], [], [], [], [], [], [], []
for user in user_indices:
    for song in song_indices:
        user_ids.append(user_indices[user])
        song_ids.append(song_indices[song])
        interactions.append(interaction_matrix[user_indices[user], song_indices[song]])
        # Map each song to its additional features
        releases.append(song_features['release'][song])
        popularities.append(song_features['popularity'][song])
        danceabilities.append(song_features['danceability'][song])
        energies.append(song_features['energy'][song])
        keys.append(song_features['key'][song])
        modes.append(song_features['mode'][song])
        valences.append(song_features['valence'][song])
        tempos.append(song_features['tempo'][song])
        time_of_days.append(song_features['time_of_day'][song])

# Create the interaction DataFrame
interaction_df = pd.DataFrame({
    'user_id': user_ids,
    'song_id': song_ids,
    'release': releases,
    'popularity': popularities,
    'danceability': danceabilities,
    'energy': energies,
    'key': keys,
    'mode': modes,
    'valence': valences,
    'tempo': tempos,
    'time_of_day': time_of_days,
    'interaction': interactions
})

interaction_df

,user_id,song_id,release,popularity,danceability,energy,key,mode,valence,tempo,time_of_day,interaction
0,0,0,-1.482494,-0.369772,-1.819800,-0.602781,-0.916059,-1.250220,-1.036510,-1.660170,-0.142363,0.000220
1,0,1,-1.482494,0.239512,0.021888,0.787531,0.482058,-1.250220,0.562468,-1.106867,1.212425,0.000714
2,0,2,0.534436,-0.796271,0.505564,-0.529349,-0.077189,-1.250220,1.456663,-0.749077,-0.142363,0.000110
3,0,3,-1.482494,-0.918128,0.629583,-0.940568,-0.077189,0.799859,-0.415419,-0.895721,-0.142363,0.000110
4,0,4,-1.482494,-0.857200,1.038847,-0.862241,-0.356813,-1.250220,0.192457,0.248418,-0.142363,0.000110
...,...,...,...,...,...,...,...,...,...,...,...,...
11516164,632,18188,0.630480,-0.065130,-0.226151,-0.940568,-1.195683,0.799859,-0.877934,0.236975,-1.497152,0.000000
11516165,632,18189,0.630480,-0.186987,0.567573,1.355406,1.041305,-1.250220,0.967719,-0.926846,-0.142363,0.000000
11516166,632,18190,0.630480,-0.369772,0.424951,1.272183,0.482058,0.799859,1.778221,0.341861,-0.142363,0.000000
11516167,632,18191,0.342347,-0.369772,-0.288160,0.748368,-1.195683,-1.250220,-0.666499,-0.400794,-0.142363,0.000000


## Data Encoding and Basic Statistics

### 1. **Encode User and Song Identifiers**:
   - `LabelEncoder` is used to transform non-numeric user and song identifiers into numeric representations. The transformed identifiers are stored in new columns `user_id` and `song_id` in the DataFrame `df`.

### 2. **Calculate Unique Counts**:
   - Calculate the number of unique users (`N`) and the number of unique songs (`M`) from the newly encoded `user_id` and `song_id` columns.

This process provides a foundational step in preparing the dataset for more complex analytical tasks, such as modeling user-song interactions in recommendation systems.


In [10]:
user_encoder = LabelEncoder()
song_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user'])
df['song_id'] = song_encoder.fit_transform(df['song'])

N = df.user_id.nunique()  # Number of users
M = df.song_id.nunique()  # Number of songs

print(N, M)
print(df.shape, interaction_df.shape)

633 18193
(68090, 15) (11516169, 12)


## Data Splitting and Exploration

### 1. **Split Data into Training and Testing Sets**:
   - The `interaction_df` DataFrame is split into training (`df_train`) and testing sets (`df_test`) using a 20% test size allocation and a random seed for reproducibility.

### 2. **Evaluate Unique Interaction Values**:
   - Determine the number of unique interaction values within the `df_train` using `interaction.nunique()` to understand the diversity of user-song interactions.

### 3. **Isolate Continuous Features**:
   - Extract continuous features (columns from the third to the second-last) from both `df_train` and `df_test` into `continuous_data_train` and `continuous_data_test` respectively.
   - Print shapes of the continuous datasets and `df_train` to understand their dimensions.

### 4. **Check for Missing Values**:
   - Calculate and display the total count of missing values per column in both `df_train` and `df_test` using `isnull().sum()` to assess data cleanliness and readiness for further processing.

In [11]:
df_train, df_test = train_test_split(interaction_df, test_size=0.2, random_state=42)
df_train

,user_id,song_id,release,popularity,danceability,energy,key,mode,valence,tempo,time_of_day,interaction
3437593,188,17309,0.438392,0.300440,0.046692,-0.475499,1.041305,0.799859,-0.208389,-0.920438,-1.497152,0.0
10363724,569,11907,0.726525,0.483225,1.441910,-0.235621,1.600552,0.799859,-1.327234,0.660358,-1.497152,0.0
6280692,345,4107,-3.979646,-1.649269,0.505564,0.860963,-1.475306,0.799859,2.223116,-0.992721,-0.142363,0.0
3531948,194,2506,-0.329963,0.178583,-0.219950,0.038525,0.482058,0.799859,-1.124608,-0.908326,-0.142363,0.0
4391169,241,6656,0.534436,-0.857200,-0.157940,0.346939,-0.356813,0.799859,-1.115799,-0.889665,1.212425,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2234489,122,14943,-1.386450,-0.796271,-0.021519,-0.759436,-0.356813,0.799859,-1.758474,-1.664642,-0.142363,0.0
4304572,236,11024,-0.329963,-0.735343,0.344338,1.174273,-0.356813,-1.250220,0.566873,-1.489303,-0.142363,0.0
10081351,554,2429,0.438392,-0.186987,0.914828,0.581922,-1.475306,0.799859,1.258443,-0.608525,-0.142363,0.0
6550634,360,1154,0.726525,-0.186987,0.697794,-0.612572,1.041305,0.799859,0.302579,-1.459622,-1.497152,0.0


In [12]:
df_train.interaction.nunique()

109

In [13]:
continuous_data_train = df_train.iloc[:, 2:-1]
continuous_data_test = df_test.iloc[:, 2:-1]
print(continuous_data_train.shape, continuous_data_test.shape, df_train.shape)
continuous_data_train

(9212935, 9) (2303234, 9) (9212935, 12)


,release,popularity,danceability,energy,key,mode,valence,tempo,time_of_day
3437593,0.438392,0.300440,0.046692,-0.475499,1.041305,0.799859,-0.208389,-0.920438,-1.497152
10363724,0.726525,0.483225,1.441910,-0.235621,1.600552,0.799859,-1.327234,0.660358,-1.497152
6280692,-3.979646,-1.649269,0.505564,0.860963,-1.475306,0.799859,2.223116,-0.992721,-0.142363
3531948,-0.329963,0.178583,-0.219950,0.038525,0.482058,0.799859,-1.124608,-0.908326,-0.142363
4391169,0.534436,-0.857200,-0.157940,0.346939,-0.356813,0.799859,-1.115799,-0.889665,1.212425
...,...,...,...,...,...,...,...,...,...
2234489,-1.386450,-0.796271,-0.021519,-0.759436,-0.356813,0.799859,-1.758474,-1.664642,-0.142363
4304572,-0.329963,-0.735343,0.344338,1.174273,-0.356813,-1.250220,0.566873,-1.489303,-0.142363
10081351,0.438392,-0.186987,0.914828,0.581922,-1.475306,0.799859,1.258443,-0.608525,-0.142363
6550634,0.726525,-0.186987,0.697794,-0.612572,1.041305,0.799859,0.302579,-1.459622,-1.497152


In [14]:
df_test.isnull().sum()

user_id         0
song_id         0
release         0
popularity      0
danceability    0
energy          0
key             0
mode            0
valence         0
tempo           0
time_of_day     0
interaction     0
dtype: int64

In [15]:
df_train.isnull().sum()

user_id         0
song_id         0
release         0
popularity      0
danceability    0
energy          0
key             0
mode            0
valence         0
tempo           0
time_of_day     0
interaction     0
dtype: int64

## Model Loading and Inspection

### 1. **Load Pre-trained Model**:
   - The Keras model is loaded from a specified path (`model_path`), where it was previously saved as `model.h5`.

### 2. **Display Model Architecture**:
   - Use `model.summary()` to print the structure of the model. This includes details of all layers, their types, outputs, and the number of parameters both trainable and non-trainable.

In [16]:
model_path = '../helpers/model.h5'
model = keras.models.load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 15)        9495        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 15)        272895      ['input_2[0][0]']                
                                                                                              

## Predictive Model Input Preparation and Execution

### 1. **Define Continuous Features**:
   - Specify the list of continuous features such as `release`, `popularity`, `danceability`, `energy`, `key`, `mode`, `valence`, `tempo`, and `time_of_day`, which are critical for the model’s input.

### 2. **Select Specific User**:
   - A specific user (`specific_user_id`) is selected from the test data to focus the predictions on.

### 3. **Prepare User Input Array**:
   - Create an array `user_input` where the selected user’s ID is repeated for each song, ensuring each song is paired with the user for prediction purposes.

### 4. **Prepare Song Input Array**:
   - Generate `song_input` as an array of indices representing all unique songs (`M` is the total count of unique songs).

### 5. **Map Song IDs to DataFrame Indices**:
   - Construct a dictionary `song_id_to_index` that maps each song ID to its corresponding index in `interaction_df` for efficient data retrieval.

### 6. **Prepare Continuous Data for Model Input**:
   - Use the mapping from song IDs to fetch and structure the continuous data corresponding to all songs, ensuring the data aligns correctly with each song ID in the input array.

### 7. **Execute Predictions**:
   - Make predictions for all songs for the selected user using the prepared inputs (`user_input`, `song_input`, and `continuous_data_input`). 

### 8. **Adjust Predictions**:
   - Normalize predicted interactions by adding the mean interaction value (`mu`) from the training set to each prediction, compensating for any baseline shifts in interaction levels.

In [17]:
user_ids_test = df_test.user_id.values
song_ids_test = df_test.song_id.values
continuous_features = ["release", "popularity", "danceability", "energy", "key", "mode", "valence", "tempo",
                       "time_of_day"]

# Select a specific user for the prediction
specific_user_id = user_ids_test[0]

# Prepare input data for the model
M = interaction_df['song_id'].nunique()  # Total number of unique songs
user_input = np.array([specific_user_id] * M)  # Repeat the user ID for each song
song_input = np.array(range(M))  # Array of all unique song IDs

# Map song IDs to indices in df
song_id_to_index = {id: idx for idx, id in enumerate(interaction_df['song_id'].unique())}

# Prepare continuous data for all songs
continuous_data_input = np.array(
    [interaction_df.loc[song_id_to_index[song_id], continuous_features] for song_id in song_input])

# Make predictions for this user with all songs, including the continuous data
predicted_interactions = model.predict([user_input, song_input, continuous_data_input])

mu = df_train.interaction.mean()  # Mean interaction value for normalization
predicted_interactions = predicted_interactions.flatten() + mu
predicted_interactions

569/569 [==============================] - 1s 1ms/step


array([-5.6552290e-06,  3.2685873e-06, -1.3613665e-06, ...,
        4.8085294e-06, -2.9832647e-06, -2.4668464e-06], dtype=float32)

## Extracting Top Song Recommendations

### 1. **Determine Number of Recommendations**:
   - Define `N` as 10 to specify the number of top recommendations to be retrieved.

### 2. **Identify Top Recommendations**:
   - Use `np.argsort()` on the `predicted_interactions` array to get indices of songs sorted by predicted interaction strength.
   - Reverse the order (`[::-1]`) to start with the highest values and select the top `N` indices (`top_n_indices`).

### 3. **Map Indices to Original Song IDs**:
   - Convert the top indices (`top_n_indices`) back to original song IDs using the `song_encoder.inverse_transform()` function, yielding `top_n_song_ids`.

In [18]:
# Determine the number of top recommendations top 10
N = 10
top_n_indices = np.argsort(predicted_interactions)[::-1][:N]

# Convert the indices to original song IDs
top_n_song_ids = song_encoder.inverse_transform(top_n_indices)

print(f"Top {N} recommended song IDs for user {specific_user_id} are:", top_n_song_ids)
print(user_input.shape)
print(song_input.shape)
print(continuous_data_input.shape)
print(df[df.user_id == specific_user_id].song.unique())

Top 10 recommended song IDs for user 277 are: ['0gnycHymXHSM6QDB' '2NC8GZPmaaNhZ9uY' '2JV7M2ql4dFHRsEu'
 '2N8dziB2ZmsefUHv' '0nWfiiw6TgdCNuYe' '0bFpFdxIfxwPU7vI'
 '0NKBY6bgxWSXytJQ' '0n8KN4mULvn0dLHt' '2LTctlkMu3f17mRp'
 '56eid8FEvdAfBOdJ']
(18193,)
(18193,)
(18193, 9)
['RCGLlYCigkopKd7D' 'wG3YQktIKVlZruUu' 'J4tQCUzKKKj3OkV0'
 'AWLAvZOhu4vLV6RA' 'b5ef6NhN3RBrQXIE' 'SAryWbXDkmkFWW3Z'
 'GQoNBkAIX2obbtPn' 'Ax8Us7NbJqe08R0a' 'Br8x0lv4QnA6pnqC'
 'ilrbbVMFVbnwmdFp' 'rPcvhlfan3sOTDX8' '6WlGmqDauWacBf9D'
 'vZ3hXeIO65MA9fBi' 's7f6VKariF11TTjK' 'aAXwzXQZdbdPxFqT'
 'L6AP1Ype1fRuWBwD' '5yY4kVyuJPdbOegL' '2HgkT8eEkxxEo2Tg'
 'bhOXqQHAGOeJqm6I' 'X8ZpmMJo8zT2QvcT' 'MwpFtrA5qYK4v5tl'
 '6U91Xztu28VOYkmU' 'NsNXlxbPuFnyPv9p' 'OBkwaSaAAVVcBaag']


## Displaying Detailed Song Information for Recommendations

### 1. **Load Song Metadata**:
   - Import song metadata from `id_information.csv` using `pd.read_csv()`, specifying a tab (`'\t'`) as the delimiter, into the DataFrame `id_information`.

### 2. **Filter Relevant Song Details**:
   - Filter `id_information` to include only the entries corresponding to the `top_n_song_ids`, which are the IDs of the top recommended songs. This is accomplished using the `isin()` method, ensuring that only relevant song information is considered.

### 3. **Display Top Recommended Songs**:
   - Print details of the top recommended songs specifically tailored for the user (`specific_user_id`). The details displayed include the artist, song title, and album name from the `recommended_songs` DataFrame.
   - This step highlights the song information, providing a more meaningful context to the recommendations, such as knowing the artist and album for each recommended song.

In [19]:
# Load song information
id_information = pd.read_csv('../data/id_information.csv', sep='\t')

# Filter id_information to only include the top N recommended song IDs
recommended_songs = id_information[id_information['id'].isin(top_n_song_ids)]

# Print the details of the top N recommended songs
print(f"Top {N} recommended songs for user {specific_user_id} are:")
print(recommended_songs[['artist', 'song', 'album_name']])

Top 10 recommended songs for user 277 are:
                  artist                            song  \
665              Rihanna                   Push Up on Me   
1093        Denzel Curry                       This Life   
1249         Dulce María                      Inevitable   
1428  Tyler, the Creator        Where This Flower Blooms   
1438                ABBA                   Dancing Queen   
4147               Sting  If I Ever Lose My Faith in You   
4197  The Rolling Stones                     Wild Horses   
4241             Placebo                   Special Needs   
4242          Anavitória               Tocando Em Frente   
9054         Tove Styrke                      Borderline   

                        album_name  
665   Good Girl Gone Bad: Reloaded  
1093                      Imperial  
1249      Extranjera Segunda Parte  
1428                    Flower Boy  
1438                       Arrival  
4147          Ten Summoner's Tales  
4197   Sticky Fingers (Remastered)  


## Analyzing and Displaying User's Recent Song History

### 1. **Filter Songs for Specific User**:
   - Extract rows from the DataFrame `df` where the `user_id` matches the specific user (`specific_user_id`). This subset contains all the songs interacted with by this particular user.

### 2. **Sort Songs by Recent Play**:
   - Sort the filtered data (`user_songs`) by the `timestamp` column in descending order to prioritize the most recent interactions. This sorted DataFrame is stored as `user_songs_sorted`.

### 3. **Identify Last 5 Played Songs**:
   - Retrieve the IDs of the last five songs played by this user from the top of the sorted DataFrame, ensuring these are the most recent songs interacted with.

### 4. **Filter Song Metadata**:
   - Use the song IDs (`last_5_song_ids`) to filter `id_information` to include only metadata for these last five songs. This step ensures that the information displayed pertains only to the most recent song interactions.

### 5. **Display Song Information**:
   - Print details about these last five songs, including the artist, song title, and album name, providing a comprehensive view of the user’s most recent music preferences.

In [20]:
# Filter the DataFrame for the specific user
user_songs = df[df.user_id == specific_user_id]

# Sort the data by the timestamp column in descending order
user_songs_sorted = user_songs.sort_values(by='timestamp', ascending=False)

last_5_song_ids = user_songs_sorted['song'].head(5).values
print(last_5_song_ids)

last_5_songs_info = id_information[id_information['id'].isin(last_5_song_ids)]
print("Information about the last 5 songs played:")
print(last_5_songs_info[['artist', 'song', 'album_name']])

['rPcvhlfan3sOTDX8' 'ilrbbVMFVbnwmdFp' 'OBkwaSaAAVVcBaag'
 'NsNXlxbPuFnyPv9p' '6U91Xztu28VOYkmU']
Information about the last 5 songs played:
              artist              song                          album_name
11445  Marty Robbins          Big Iron  Gunfighter Ballads And Trail Songs
42102  Marty Robbins     Billy the Kid  Gunfighter Ballads And Trail Songs
42687  Marty Robbins        Utah Carol  Gunfighter Ballads And Trail Songs
78856      blink-182  When I Was Young                    Dogs Eating Dogs
94075      blink-182        Boxing Day                    Dogs Eating Dogs


## Similarity Calculation for Recommended Songs

### 1. **Function for Similarity Calculation**:
   - The `calculate_similarity` function computes the cosine similarity between a target song's feature vector and a set of song features. It returns the similarity scores from a 2D array to a 1D array for easier manipulation.

### 2. **Initialize Variables**:
   - Set `maxi` to -1 to track the highest similarity score found.
   - Set `best_idx` to 0 to store the index of the song with the highest similarity.

### 3. **Iterate Over Top Recommended Songs**:
   - Loop through the first 10 recommended song IDs (`top_n_song_ids`).
   - For each recommended song:
     - Extract the feature vector from `interaction_df` using the song's index derived from `song_indices`.

### 4. **Fetch Features of Last 5 Played Songs**:
   - Construct an array of feature vectors for the last five played songs, again using indices from `interaction_df`.

### 5. **Calculate Similarity for Each Recommended Song**:
   - For each recommended song, compute its similarity to each of the last five played songs using the previously defined `calculate_similarity` function.
   - Sum the similarity scores to get an overall similarity measure for the recommended song against all last played songs.

### 6. **Identify Song with Highest Similarity**:
   - Track and update the maximum similarity score (`maxi`) and the corresponding index (`best_idx`) if the current song's summed similarity score exceeds the previous maximum.

This method effectively determines which of the top recommended songs are most similar to the user's recent listening habits.


In [21]:
def calculate_similarity(target_song_features, songs_features):
    similarity = cosine_similarity([target_song_features], songs_features)
    return similarity[0]  # similarity[0] because the result is in a 2D array

maxi = -1
best_idx = 0

In [22]:
for i in range(10):
    recommended_song_id = top_n_song_ids[i]
    # Get the feature vector for the recommended song
    recommended_song_features = interaction_df.loc[interaction_df.song_id[song_indices[recommended_song_id]]]

    # Get the feature vectors for the last 5 played songs
    last_5_songs_features = np.array(
        [interaction_df.loc[interaction_df.song_id[song_indices[song_id]]] for song_id in last_5_song_ids])

    # Calculate similarity
    similarities = calculate_similarity(np.array(recommended_song_features).reshape(1, -1)[0], last_5_songs_features)
    print(np.sum(similarities), end="\n\n\n")

    if np.sum(similarities) > maxi:
        maxi = np.sum(similarities)
        best_idx = i

4.999998764571219


4.9999989531642


4.999999596224476


4.999997758815648


4.9999994716710745


4.999999259204033


4.999999468043022


4.999994443006962


4.999996592584553


4.999999309123432




## Generating and Comparing Predicted Recommendations with Actual Preferences

### 1. **Define Function to Retrieve Actual Relevant Songs**:
   - The `get_relevant_songs` function is designed to fetch the unique song IDs that a specific user has interacted with from the DataFrame `df`. This establishes a baseline of songs that are known to be relevant to the user.

### 2. **Initialize Prediction and Actual Dictionaries**:
   - Two dictionaries, `predictions` and `actual`, are initialized to store the predicted top song IDs and actual relevant song IDs for each user, respectively.

### 3. **Iterate Over a Subset of Users**:
   - Loop through each user ID in the test set (limited to the first 10 users for demonstration). This looping facilitates the prediction and validation process for multiple users in a manageable subset.

### 4. **Generate Predictions for Each User**:
   - For each user:
     - Create an input array (`user_input`) that repeats the user ID for each song, corresponding to the total number of unique songs (`M`).
     - Prepare `continuous_data_input` by collecting the continuous feature data for all songs relevant to the current user input.
     - Predict interaction scores using the model for all songs with the prepared inputs. Flatten the result to simplify handling.
     - Sort the predicted scores in descending order and extract the indices of the top `N` scores.
     - Use `song_encoder.inverse_transform` to convert these indices back into original song IDs (`top_n_song_ids`).

### 5. **Store Predictions and Actual Song IDs**:
   - Store the top `N` predicted song IDs for each user in the `predictions` dictionary.
   - Fetch and store the actual relevant songs for the user using `get_relevant_songs` and store them in the `actual` dictionary.

### 6. **Output Progress**:
   - Print the total number of users being processed and the current progress after each user's data is processed to monitor the computation and ensure it is proceeding correctly.

In [23]:
np.unique(user_ids_test)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [24]:
def get_relevant_songs(user_id, df):
    return df[df.user_id == user_id]['song'].unique()

predictions = {}
actual = {}

i = 0
# Get predictions and actual relevant songs for each user
for user_id in np.unique(user_ids_test):
    # Predict top N songs
    user_input = np.array([user_id] * M)
    continuous_data_input = np.array(
        [interaction_df.loc[song_id_to_index[song_id], continuous_features] for song_id in song_input])
    predicted_interactions = model.predict([user_input, song_input, continuous_data_input]).flatten()
    top_n_indices = np.argsort(predicted_interactions)[::-1][:N]
    top_n_song_ids = song_encoder.inverse_transform(top_n_indices)

    # Store the top N song IDs for each user
    predictions[user_id] = top_n_song_ids.tolist()

    # Get actual relevant songs for the user
    actual_relevant_songs = get_relevant_songs(user_id, df)
    actual[user_id] = actual_relevant_songs.tolist()
    i += 1
    print(len(user_ids_test), i)


569/569 [==============================] - 1s 1ms/step
2303234 1
569/569 [==============================] - 1s 1ms/step
2303234 2
569/569 [==============================] - 1s 1ms/step
2303234 3
569/569 [==============================] - 1s 1ms/step
2303234 4
569/569 [==============================] - 1s 1ms/step
2303234 5
569/569 [==============================] - 1s 1ms/step
2303234 6
569/569 [==============================] - 1s 1ms/step
2303234 7
569/569 [==============================] - 1s 1ms/step
2303234 8
569/569 [==============================] - 1s 1ms/step
2303234 9
569/569 [==============================] - 1s 1ms/step
2303234 10
569/569 [==============================] - 1s 1ms/step
2303234 11
569/569 [==============================] - 1s 1ms/step
2303234 12
569/569 [==============================] - 1s 1ms/step
2303234 13
569/569 [==============================] - 1s 1ms/step
2303234 14
569/569 [==============================] - 1s 1ms/step
2303234 15
569/569 [==========

2303234 124
569/569 [==============================] - 1s 1ms/step
2303234 125
569/569 [==============================] - 1s 1ms/step
2303234 126
569/569 [==============================] - 1s 1ms/step
2303234 127
569/569 [==============================] - 1s 1ms/step
2303234 128
569/569 [==============================] - 1s 1ms/step
2303234 129
569/569 [==============================] - 1s 1ms/step
2303234 130
569/569 [==============================] - 1s 1ms/step
2303234 131
569/569 [==============================] - 1s 1ms/step
2303234 132
569/569 [==============================] - 1s 1ms/step
2303234 133
569/569 [==============================] - 1s 1ms/step
2303234 134
569/569 [==============================] - 1s 1ms/step
2303234 135
569/569 [==============================] - 1s 1ms/step
2303234 136
569/569 [==============================] - 1s 1ms/step
2303234 137
569/569 [==============================] - 1s 1ms/step
2303234 138
569/569 [==============================] - 1s 1ms/

569/569 [==============================] - 1s 1ms/step
2303234 247
569/569 [==============================] - 1s 1ms/step
2303234 248
569/569 [==============================] - 1s 1ms/step
2303234 249
569/569 [==============================] - 1s 1ms/step
2303234 250
569/569 [==============================] - 1s 1ms/step
2303234 251
569/569 [==============================] - 1s 1ms/step
2303234 252
569/569 [==============================] - 1s 1ms/step
2303234 253
569/569 [==============================] - 1s 1ms/step
2303234 254
569/569 [==============================] - 1s 1ms/step
2303234 255
569/569 [==============================] - 1s 2ms/step
2303234 256
569/569 [==============================] - 1s 2ms/step
2303234 257
569/569 [==============================] - 1s 1ms/step
2303234 258
569/569 [==============================] - 1s 1ms/step
2303234 259
569/569 [==============================] - 1s 1ms/step
2303234 260
569/569 [==============================] - 1s 1ms/step
2303234

569/569 [==============================] - 1s 1ms/step
2303234 369
569/569 [==============================] - 1s 1ms/step
2303234 370
569/569 [==============================] - 1s 1ms/step
2303234 371
569/569 [==============================] - 1s 1ms/step
2303234 372
569/569 [==============================] - 1s 1ms/step
2303234 373
569/569 [==============================] - 1s 1ms/step
2303234 374
569/569 [==============================] - 1s 1ms/step
2303234 375
569/569 [==============================] - 1s 1ms/step
2303234 376
569/569 [==============================] - 1s 1ms/step
2303234 377
569/569 [==============================] - 1s 1ms/step
2303234 378
569/569 [==============================] - 1s 1ms/step
2303234 379
569/569 [==============================] - 1s 1ms/step
2303234 380
569/569 [==============================] - 1s 1ms/step
2303234 381
569/569 [==============================] - 1s 1ms/step
2303234 382
569/569 [==============================] - 1s 1ms/step
2303234

569/569 [==============================] - 1s 1ms/step
2303234 491
569/569 [==============================] - 1s 1ms/step
2303234 492
569/569 [==============================] - 1s 1ms/step
2303234 493
569/569 [==============================] - 1s 1ms/step
2303234 494
569/569 [==============================] - 1s 1ms/step
2303234 495
569/569 [==============================] - 1s 1ms/step
2303234 496
569/569 [==============================] - 1s 1ms/step
2303234 497
569/569 [==============================] - 1s 1ms/step
2303234 498
569/569 [==============================] - 1s 1ms/step
2303234 499
569/569 [==============================] - 1s 1ms/step
2303234 500
569/569 [==============================] - 1s 1ms/step
2303234 501
569/569 [==============================] - 1s 1ms/step
2303234 502
569/569 [==============================] - 1s 1ms/step
2303234 503
569/569 [==============================] - 1s 1ms/step
2303234 504
569/569 [==============================] - 1s 1ms/step
2303234

569/569 [==============================] - 1s 1ms/step
2303234 613
569/569 [==============================] - 1s 1ms/step
2303234 614
569/569 [==============================] - 1s 1ms/step
2303234 615
569/569 [==============================] - 1s 1ms/step
2303234 616
569/569 [==============================] - 1s 1ms/step
2303234 617
569/569 [==============================] - 1s 1ms/step
2303234 618
569/569 [==============================] - 1s 1ms/step
2303234 619
569/569 [==============================] - 1s 1ms/step
2303234 620
569/569 [==============================] - 1s 1ms/step
2303234 621
569/569 [==============================] - 1s 1ms/step
2303234 622
569/569 [==============================] - 1s 1ms/step
2303234 623
569/569 [==============================] - 1s 1ms/step
2303234 624
569/569 [==============================] - 1s 1ms/step
2303234 625
569/569 [==============================] - 1s 1ms/step
2303234 626
569/569 [==============================] - 1s 1ms/step
2303234

## Recommendation System Evaluation Metrics

### Metric Calculations

#### 1. **Precision at K**:
   - Measures the proportion of recommended items in the top-K set that are relevant.

#### 2. **Recall at K**:
   - Assesses how many relevant items are found in the top-K recommendations.

#### 3. **Mean Average Precision at K (MAP@K)**:
   - Computes the mean of the average precision scores for each user, considering only the top-K recommendations.

#### 4. **Mean Reciprocal Rank (MRR)**:
   - Calculates the average of the reciprocal of the rank of the first relevant item among the recommendations.

#### 5. **Normalized Discounted Cumulative Gain at K (NDCG@K)**:
   - Evaluates the gain of a recommendation based on its position in the result list, giving higher importance to hits at top ranks.

### Functions Defined

- **`precision_at_k`**: Compares the top-K predicted items to the actual relevant items for each user to calculate precision.
- **`recall_at_k`**: Identifies how many of the relevant items appear in the top-K predictions for each user.
- **`mean_avg_precision_at_k`** and **`mean_average_precision_at_k`**: Both calculate the average precision at K for predictions against actual data.
- **`mean_reciprocal_rank`**: Computes the average reciprocal rank where the rank is the position of the first relevant recommendation.
- **`dcg_at_k`**: Computes the Discounted Cumulative Gain at K, a measure of ranking quality.
- **`ndcg_at_k`**: Normalizes the DCG at K by the ideal or perfect DCG at K, providing a measure of the model's performance relative to the best possible scenario.


In [25]:
def precision_at_k(actual, predicted, k):
    precision_scores = []
    for user_id in actual:
        # Initialize true positives count
        true_positives = 0
        # Check if user exists in predictions
        if user_id in predicted and len(predicted[user_id]) >= k:
            # Count the number of relevant items in the top k predictions
            true_positives = len(set(predicted[user_id][:k]) & set(actual[user_id]))
        # Calculate precision for this user
        precision = true_positives / float(k)
        precision_scores.append(precision)
    # Return the average precision at k for all users
    return sum(precision_scores) / len(precision_scores)


def recall_at_k(actual, predicted, k):
    recall_scores = []
    for user_id in actual:
        # Initialize true positives count
        true_positives = 0
        # Check if user exists in predictions
        if user_id in predicted:
            # Count the number of relevant items in the top k predictions
            true_positives = len(set(predicted[user_id][:k]) & set(actual[user_id]))
            recall = true_positives / float(len(actual[user_id]))
        else:
            # If no predictions for the user, recall is 0
            recall = 0.0
        recall_scores.append(recall)
    # Return the average recall at k for all users
    return sum(recall_scores) / len(recall_scores)


def avg_precision_at_k(actual, predicted, k=10):
    ap_sum = 0
    for user, true_items in actual.items():
        pred_items = predicted[user][:k]
        hits = 0
        sum_precs = 0
        for i, p in enumerate(pred_items):
            if p in true_items:
                hits += 1
                sum_precs += hits / (i + 1.0)
        ap_sum += sum_precs / min(len(true_items), k)
    return ap_sum / len(actual)


def mean_avg_precision_at_k(actual, predicted, k=10):
    return avg_precision_at_k(actual, predicted, k)


def mean_average_precision_at_k(actual, predicted, k=10):
    AP_sum = 0.0
    for user_id in actual:
        if user_id in predicted:
            pred_items = predicted[user_id][:k]
            hits = 0
            sum_precisions = 0
            for i, p in enumerate(pred_items):
                if p in actual[user_id] and p not in pred_items[:i]:
                    hits += 1
                    sum_precisions += hits / (i + 1.0)
            AP_sum += sum_precisions / min(len(actual[user_id]), k)
    return AP_sum / len(actual)


def mean_reciprocal_rank(actual, predicted):
    MRR_sum = 0.0
    for user_id in actual:
        if user_id in predicted:
            pred_items = predicted[user_id]
            for rank, p in enumerate(pred_items, start=1):
                if p in actual[user_id]:
                    MRR_sum += 1.0 / rank
                    break
    return MRR_sum / len(actual)


def dcg_at_k(relevances, k):
    relevances = np.asfarray(relevances)[:k]
    if relevances.size:
        return np.sum(relevances / np.log2(np.arange(2, relevances.size + 2)))
    return 0.0


def ndcg_at_k(actual, predicted, k=10):
    NDCG_sum = 0.0
    for user_id in actual:
        if user_id in predicted:
            pred_items = predicted[user_id][:k]
            true_relevances = [1 if item in actual[user_id] else 0 for item in pred_items]
            ideal_relevances = [1] * len(actual[user_id])
            NDCG_sum += dcg_at_k(true_relevances, k) / dcg_at_k(ideal_relevances, k)
    return NDCG_sum / len(actual)

k = 10
precision = precision_at_k(actual, predictions, k)
recall = recall_at_k(actual, predictions, k)
map_k = mean_average_precision_at_k(actual, predictions, k)
mrr = mean_reciprocal_rank(actual, predictions)
ndcg_k = ndcg_at_k(actual, predictions, k)

print(f"Precision@{k}: {precision}")
print(f"Recall@{k}: {recall}")
print(f"MAP@{k}: {map_k}")
print(f"MRR: {mrr}")
print(f"NDCG@{k}: {ndcg_k}")

Precision@10: 0.004739336492890997
Recall@10: 0.0014122836746558352
MAP@10: 0.0010522455427668697
MRR: 0.009890543895283235
NDCG@10: 0.004212361192336921
